In [8]:


import gym
import numpy as np
env = gym.make('LunarLander-v2')

import keras
from keras import models, layers



def initiate_NN_single(learning_rate):
    optimizer = keras.optimizers.Adam(lr=0.01)
    optimizer.learning_rate.assign(learning_rate)
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_dim=8))
    model.add(layers.Dense(4, activation='linear'))
    model.compile(loss='mse',optimizer=optimizer)
    return model

def deep_Sarsa(model,gamma = 0.99,decay_rate = 0.002,max_e=1,min_e=0.0005,max_iteration=5000):
    total_rewards=[]
    running_rewards=[]
    for iteration in range(5000):
        state = env.reset()
        total_reward=0
        epsilon = min_e + (max_e - min_e)*np.exp(-decay_rate*iteration)
        done=False

        random_value = np.random.random()
        if random_value<=epsilon:
            action = env.action_space.sample()
        else:
            Y=model.predict(state.reshape(1,-1))
            action = Y[0].tolist().index(np.max(Y))

        while not done:
            new_state, reward, done, info = env.step(action)
            total_reward+=reward
            random_value = np.random.random()
            if random_value<=epsilon:
                new_action = env.action_space.sample()
            else:
                Y=model.predict(new_state.reshape(1,-1))
                new_action = Y[0].tolist().index(np.max(Y))
            if done:
                target = reward
            else:
                Y=model.predict(new_state.reshape(1,-1))
                target = reward +gamma*Y[0][new_action]
            X=state.reshape(1,-1)
            Y=model.predict(X)
            Y[0][action] = target
            model.fit(X, np.array(Y).reshape(1,-1),verbose=0)
            state = new_state
            action =new_action
        total_rewards.append(total_reward)
        try:
            running_rewards.append(np.average(total_rewards[-100:]))
            print('episode: ', iteration, ' score: ', '%.2f' % total_reward, ' avg_score: ', '%.2f' % np.average(total_rewards[-100:]),'%.5f' % epsilon)
        except:
            print('episode: ', iteration, ' score: ', '%.2f' % total_reward, ' avg_score: ', '%.2f' % np.average(total_rewards),'%.5f' % epsilon)
        if np.sum(np.array(running_rewards[-20:])>200)>18 and epsilon<0.1 :
            break
        return model,total_rewards,running_rewards

    
if __name__=='__main__':    
    learning_rate=1
    gamma = 0.99
    decay_rate = 0.002
    max_e=1
    min_e=0.0005
    model = initiate_NN_single(learning_rate)
    model,total_reward,running_reward = deep_Sarsa(model,gamma,decay_rate,max_e,min_e,max_iteration=1)
    model.save_weights('weights/DSarsa_test_'+str(max_e)+'-'+str(min_e)+'-'+str(decay_rate)\
                         +'-'+str(gamma)+'-'+str(learning_rate)+'.h5')
    import pickle
    with open('weights/DSarsa_test_'+str(max_e)+'-'+str(min_e)+'-'+str(decay_rate )+'-'+str(gamma)+'-'+str(learning_rate)+'.json', 'wb') as fp:
        pickle.dump({'rewards':total_reward,'rolling_reward':running_reward}, fp)

episode:  0  score:  -140.48  avg_score:  -140.48 1.00000
